# Unsupervised Analysis of days of Week

Treating crossings each day as features to learn about the relationships between various days.

In [1]:
import pandas as pd
import numpy as np

## Get Data 

In [2]:
df = pd.read_csv('Fremont_Bridge_Bicycle_Counter_20240716.csv', index_col ='Date', parse_dates=True)

C:\Users\alish\AppData\Local\Temp\ipykernel_20432\3922072062.py:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df = pd.read_csv('Fremont_Bridge_Bicycle_Counter_20240716.csv', index_col ='Date', parse_dates=True)


In [3]:
np.unique(df.index.time)

array([datetime.time(0, 0), datetime.time(1, 0), datetime.time(2, 0),
       datetime.time(3, 0), datetime.time(4, 0), datetime.time(5, 0),
       datetime.time(6, 0), datetime.time(7, 0), datetime.time(8, 0),
       datetime.time(9, 0), datetime.time(10, 0), datetime.time(11, 0),
       datetime.time(12, 0), datetime.time(13, 0), datetime.time(14, 0),
       datetime.time(15, 0), datetime.time(16, 0), datetime.time(17, 0),
       datetime.time(18, 0), datetime.time(19, 0), datetime.time(20, 0),
       datetime.time(21, 0), datetime.time(22, 0), datetime.time(23, 0)],
      dtype=object)

In [4]:
! head Fremont_Bridge_Bicycle_Counter_20240716.csv

Date,"Fremont Bridge Sidewalks, south of N 34th St","Fremont Bridge Sidewalks, south of N 34th St Cyclist West Sidewalk","Fremont Bridge Sidewalks, south of N 34th St Cyclist East Sidewalk"
10/02/2012 01:00:00 PM,55,7,48
10/02/2012 02:00:00 PM,130,55,75
10/02/2012 03:00:00 PM,152,81,71
10/02/2012 04:00:00 PM,278,167,111
10/02/2012 05:00:00 PM,563,393,170
10/02/2012 06:00:00 PM,381,236,145
10/02/2012 07:00:00 PM,175,104,71
10/02/2012 08:00:00 PM,86,51,35
10/02/2012 09:00:00 PM,63,35,28


In [5]:
df.head()

,"Fremont Bridge Sidewalks, south of N 34th St","Fremont Bridge Sidewalks, south of N 34th St Cyclist West Sidewalk","Fremont Bridge Sidewalks, south of N 34th St Cyclist East Sidewalk"
Date,,,
2012-10-02 13:00:00,55.0,7.0,48.0
2012-10-02 14:00:00,130.0,55.0,75.0
2012-10-02 15:00:00,152.0,81.0,71.0
2012-10-02 16:00:00,278.0,167.0,111.0
2012-10-02 17:00:00,563.0,393.0,170.0


In [7]:
df.shape

(102947, 3)

In [ ]:
df.index

In [ ]:
pd.to_datetime(df.index, format = '%m/%d/%y %H:%M:%S %p')

In [ ]:
df.plot()

In [ ]:
df.resample('W').sum().plot();

In [ ]:
import matplotlib.pyplot as plt
df.resample('W').sum().plot()

In [ ]:
df['Total'] = df['Fremont Bridge Sidewalks, south of N 34th St'] + df['Fremont Bridge Sidewalks, south of N 34th St Cyclist West Sidewalk'] + df['Fremont Bridge Sidewalks, south of N 34th St Cyclist East Sidewalk']
ax = df.resample('D').sum().rolling(365).sum().plot()
ax.set_ylim(0,None)

In [ ]:
df.groupby(df.index.time).mean().plot()

In [ ]:
pivoted = df.pivot_table('Total', index=df.index.time, columns=df.index.date)
pivoted.iloc[:5,:5]

In [ ]:
pivoted.plot(legend=False, alpha=0.01)

## Principal Component Analysis

In [ ]:
pivoted.index

In [ ]:
X = pivoted.fillna(0).T.values
X.shape

In [ ]:
from sklearn.decomposition import PCA
X2 = PCA(2, svd_solver='full').fit_transform(X)

In [ ]:
X2.shape

In [ ]:
import matplotlib.pyplot as plt
plt.scatter(X2[:,0], X2[:,1])

## Unsupervised Clustering 

In [ ]:
from sklearn.mixture import GaussianMixture
gmm = GaussianMixture(2)
gmm.fit(X)
labels = gmm.predict(X)
labels

In [ ]:
plt.scatter(X2[:,0], X2[:,1],c=labels, cmap= 'rainbow')
plt.colorbar()

In [ ]:
pivoted.T[labels == 1].T.plot(legend=False, alpha=0.1)
pivoted.T[labels == 0].T.plot(legend=False, alpha =0.1)

## Comparing With Day of Week

In [ ]:
dayofweek = pd.DatetimeIndex(pivoted.columns).dayofweek

In [ ]:
plt.scatter(X2[:,0], X2[:,1], c = dayofweek, cmap = 'rainbow')
plt.colorbar()

## Analyzing Outliers 

The following points are week days with a holiday like pattern

In [ ]:
dates = pd.DatetimeIndex(pivoted.columns)
dates[(labels == 1) & (dayofweek < 5)]

>>> What's up with Feb 6, 2017? [Snow Storm](https://www.seattletimes.com/seattle-news/weather/weather-service-predicts-3-to-6-inches-of-snow-in-seattle-area/)

In [ ]:
[Seattle Transportation Website](https://data.seattle.gov/Transportation/Fremont-Bridge-Bicycle-Counter/65db-xm6k/about_data)